In [1]:
# Importing the Neccessary Libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     RandomizedSearchCV,
                                     GridSearchCV)

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (accuracy_score,
                             f1_score,
                             precision_score,
                             recall_score,
                             RocCurveDisplay,
                             precision_recall_curve,
                             average_precision_score,
                             roc_auc_score,
                             roc_curve, auc)

from sklearn.neighbors import NearestCentroid
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Loading the Dataframe

test_data = pd.read_excel('/content/test_data.xlsx')
train_data = pd.read_excel('/content/train_data.xlsx')

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib

class LoanDefaultModel:
    def __init__(self, model_type="random_forest"):
        if model_type == "random_forest":
            self.model = RandomForestClassifier(random_state=42)
        elif model_type == "logistic_regression":
            self.model = LogisticRegression(max_iter=1000)
        else:
            raise ValueError("Unsupported model type")

    def load_data(self, path):
        self.data = pd.read_excel(path)

    def preprocess(self):
        # Example preprocessing: handle missing values, encode categorical variables, etc.
        # Instead of dropping rows with NA, fill NA for issue_d column
        if 'issue_d' in self.data.columns:
            self.data['issue_d'] = self.data['issue_d'].fillna(self.data['issue_d'].mode()[0])
            self.data['issue_d'] = pd.to_datetime(self.data['issue_d'])
            self.data['issue_d_year'] = self.data['issue_d'].dt.year
            self.data['issue_d_month'] = self.data['issue_d'].dt.month
            self.data['issue_d_day'] = self.data['issue_d'].dt.day
            # Remove the original 'issue_d' column only once
            self.data = self.data.drop('issue_d', axis=1) # This line drops the 'issue_d' column
        else:
            print("Warning: 'issue_d' column not found in the DataFrame. Skipping preprocessing for this column.")

        # Removed the redundant drop statement for 'issue_d' here

        # One-hot encode categorical features (excluding date columns)
        categorical_cols = self.data.select_dtypes(include=['object']).columns
        self.data = pd.get_dummies(self.data, columns=categorical_cols, drop_first=True)

        # Convert datetime columns to numerical features (ordinal encoding for dates)
        for col in self.data.select_dtypes(include=['datetime64']).columns:
            self.data[col] = self.data[col].apply(lambda date: date.toordinal())

        # Split features and target
        self.X = self.data.drop("loan_status", axis=1)
        self.y = self.data["loan_status"]
        print("Shape of X: \n", self.X.head(5))

        # Train-test split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

    def train(self):
        self.model.fit(self.X_train, self.y_train)

    def test(self):
        y_pred = self.model.predict(self.X_test)
        print("Classification Report:")
        print(classification_report(self.y_test, y_pred))
        print(f"Accuracy: {accuracy_score(self.y_test, y_pred):.2f}")

    def predict(self, input_data):
        input_df = pd.DataFrame([input_data])
        prediction = self.model.predict(input_df)
        return prediction

    def save_model(self, filename):
        joblib.dump(self.model, filename)

    def load_model(self, filename):
        self.model = joblib.load(filename)

if __name__ == "__main__":
    # Initialize and run the model pipeline
    model = LoanDefaultModel(model_type="random_forest")
    model.load_data("train_data.xlsx")
    model.preprocess()
    model.train()
    model.test()
    model.save_model("loan_default_model.pkl")


Shape of X: 
    customer_id  transaction_date  cibil_score  total_no_of_acc  annual_inc  \
0     10608026            735234          665                9     70000.0   
1     10235120            735234          660                8     65000.0   
2     10705805            735234          660                7     73000.0   
3     11044991            735234          690                5    118000.0   
4     10161054            735234          665                5     63000.0   

   int_rate  loan_amnt  installment  account_bal  emp_length  ...  \
0     16.24       7200       253.99         4648          11  ...   
1     23.40       6000       233.52        14051          11  ...   
2     17.57       8000       287.50        14885          11  ...   
3     12.85      10000       336.22         7542           2  ...   
4     14.98      10000       237.80         6318          11  ...   

   purpose_medical  purpose_moving  purpose_other  purpose_renewable_energy  \
0            False     


The model achieves 76% accuracy, performing well in identifying defaulters (class 1) with 93% recall and 79% precision, ensuring most flagged defaults are correct. However, it struggles with non-defaulters (class 0), with only 29% recall, leading to many missed non-default predictions. The imbalance in the dataset likely skews performance toward defaults. The weighted F1-score of 0.73 reflects better performance for defaults than non-defaults. To improve, handle class imbalance using oversampling, undersampling, or class-weight adjustments. Refining features, tuning hyperparameters, and adjusting thresholds can balance precision-recall trade-offs. This is critical for approving low-risk loans while minimizing defaults.
